In [110]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

engine = create_engine('sqlite:///MyDB.sqlite3', echo=True)

data = pd.read_sql("select level, hot,num, price, area from item;", con=engine)
data = data[(data['level'] != '')]

data["area"] = data["area"].apply(lambda x: x.replace("[", "").replace("]", ""))
data["province"] = data["area"].apply(lambda x: x.split("·")[0])
city = list(set(data['province']))
data['province'] = data['province'].apply(lambda x:city.index(x))

train_data = data[['price', 'num', 'hot', 'province']]

from sklearn.preprocessing import scale
import numpy as np

train_data = scale(train_data)

y = data[['level']]

from imblearn.over_sampling import  SMOTE

train_data, y =  SMOTE().fit_sample(train_data, y)

x_train,x_test,y_train,y_test = train_test_split(train_data,y,test_size=0.2, stratify=y)

2018-10-25 19:43:26,090 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-10-25 19:43:26,091 INFO sqlalchemy.engine.base.Engine ()
2018-10-25 19:43:26,093 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-10-25 19:43:26,095 INFO sqlalchemy.engine.base.Engine ()
2018-10-25 19:43:26,097 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("select level, hot,num, price, area from item;")
2018-10-25 19:43:26,099 INFO sqlalchemy.engine.base.Engine ()
2018-10-25 19:43:26,101 INFO sqlalchemy.engine.base.Engine select level, hot,num, price, area from item;
2018-10-25 19:43:26,102 INFO sqlalchemy.engine.base.Engine ()


/home/miku/anaconda2/envs/sun/lib/python3.5/site-packages/ipykernel_launcher.py:22: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
/home/miku/anaconda2/envs/sun/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/miku/anaconda2/envs/sun/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [113]:
rf = RandomForestClassifier(n_estimators=100,oob_score=True, n_jobs=1)    # max_depth=10最大树深,
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
score = rf.score(x_test,y_test)
print(score)

0.7400497512437811


In [114]:
from sklearn.metrics import classification_report

"""
精确度: 表示的是预测为正的样本中有多少是真正的正样本。
召回率:表示的是样本中的正例有多少被预测正确了。
F1值:2*精度*召回率/(精度+召回率)
支持数: 该类在样本中出现的总次数
"""

print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

        3A景区       0.75      0.81      0.78       268
        4A景区       0.69      0.60      0.64       268
        5A景区       0.77      0.82      0.79       268

   micro avg       0.74      0.74      0.74       804
   macro avg       0.74      0.74      0.74       804
weighted avg       0.74      0.74      0.74       804



In [115]:
from sklearn.externals import joblib

joblib.dump(rf, 'random_forest.pkl')

['random_forest.pkl']